In [1]:
import pandas as pd

df = pd.read_csv("./data/subject-info.csv", sep=";", encoding="utf-8")
df.head()

,Patient ID,Follow-up period from enrollment (days),days_4years,Exit of the study,Cause of death,Age,Gender (male=1),Weight (kg),Height (cm),Body Mass Index (Kg/m2),...,Angiotensin-II receptor blocker (yes=1),Anticoagulants/antitrombotics (yes=1),Betablockers (yes=1),Digoxin (yes=1),Loop diuretics (yes=1),Spironolactone (yes=1),Statins (yes=1),Hidralazina (yes=1),ACE inhibitor (yes=1),Nitrovasodilator (yes=1)
0,P0001,2065,1460,NaN,0,58,1,83,163,"31,2",...,0,1,1,1,1,0,0,0,1,0
1,P0002,2045,1460,NaN,0,58,1,74,160,"28,9",...,1,1,1,0,0,0,1,0,0,0
2,P0003,2044,1460,NaN,0,69,1,83,174,"27,4",...,1,1,1,1,1,0,0,0,0,0
3,P0004,2044,1460,NaN,0,56,0,84,165,"30,9",...,1,1,1,0,1,1,0,0,0,0
4,P0005,2043,1460,NaN,0,70,1,97,183,29,...,0,1,1,0,1,0,1,0,1,1


In [7]:
for col in df.columns:
    print(col)

Patient ID
Follow-up period from enrollment (days)
days_4years
Exit of the study
Cause of death
Age
Gender (male=1)
Weight (kg)
Height (cm)
Body Mass Index (Kg/m2)
NYHA class
Diastolic blood  pressure (mmHg)
Systolic blood pressure (mmHg)
HF etiology - Diagnosis
Diabetes (yes=1)
History of dyslipemia (yes=1)
Peripheral vascular disease (yes=1)
History of hypertension (yes=1)
Prior Myocardial Infarction (yes=1)
Prior implantable device
Prior Revascularization
Syncope
daily smoking (cigarretes/day)
smoke-free time (years)
cigarettes /year
alcohol consumption (standard units)
Albumin (g/L)
ALT or GPT (IU/L)
AST or GOT (IU/L)
Normalized Troponin
Total Cholesterol (mmol/L)
Creatinine (?mol/L)
Gamma-glutamil transpeptidase (IU/L)
Glucose (mmol/L)
Hemoglobin (g/L)
HDL (mmol/L)
Potassium (mEq/L)
LDL (mmol/L)
Sodium (mEq/L)
Pro-BNP (ng/L)
Protein (g/L)
T3 (pg/dL)
T4 (ng/L)
Troponin (ng/mL)
TSH (mIU/L)
Urea (mg/dL)
Signs of pulmonary venous hypertension (yes=1)
Cardiothoracic ratio
Left atrial s

In [2]:
import pandas as pd
import numpy as np

# 🔹 Funktion zur vollständigen Datenbereinigung
def clean_data(df):
    print("\n🔷 Starte Datenbereinigung...")

    # 🔹 1. Entferne doppelte Spaltennamen, falls vorhanden
    df = df.loc[:, ~df.columns.duplicated()].copy()

    # 🔹 2. Falls 'Average_RR_ms' zwei Spalten enthält, korrekt aufteilen
    if "Average_RR_ms" in df.columns:
        if isinstance(df["Average_RR_ms"], pd.DataFrame):
            df["Average_RR_1_ms"] = df["Average_RR_ms"].iloc[:, 0]
            df["Average_RR_2_ms"] = df["Average_RR_ms"].iloc[:, 1]
            df.drop(columns=["Average_RR_ms"], inplace=True)
            print("✅ 'Average_RR_ms' erfolgreich aufgeteilt in zwei Spalten!")

    # 🔹 3. Problematische numerische Spalten bereinigen
    problematic_columns = df.select_dtypes(include=['object']).columns

    # 🔹 4. Entferne nicht-numerische Spalten aus der Konvertierungsliste
    exclude_columns = ["Patient_ID", "Holter_onset_hhmmss"]
    convertible_columns = [col for col in problematic_columns if col not in exclude_columns]


    # 🔹 5. Ersetze Kommas durch Punkte und konvertiere nur echte numerische Werte
    # 🔹 Problematische Zeichen in numerischen Spalten entfernen und umwandeln
    for col in convertible_columns:
        if df[col].dtype == "object":  # Sicherstellen, dass die Spalte Strings sind
            try:
                df[col] = df[col].astype(str)  # Sicherstellen, dass es Strings sind
                df[col] = df[col].str.replace(",", ".", regex=True)  # Kommas durch Punkte ersetzen
                df[col] = df[col].str.replace(r"[^0-9.]", "", regex=True)  # Entferne nicht-numerische Zeichen außer Punkten
                df[col] = pd.to_numeric(df[col], errors="coerce")  # Konvertiere zu Float, setze Fehler auf NaN
            except ValueError:
                print(f"⚠️ Spalte {col} konnte nicht konvertiert werden und bleibt im Originalformat.")
        else:
            print(f"ℹ️ Spalte {col} ist bereits numerisch und wird übersprungen.")

    print(f"✅ Erfolgreich konvertierte Spalten: {convertible_columns}")
    print(f"⚠️ Nicht konvertierte Spalten (enthielten Text): {exclude_columns}")


    # 🔹 6. Fehlende Werte behandeln
    missing_values = df.isnull().sum()
    high_missing = missing_values[missing_values > 0.3 * len(df)].index.tolist()
    print(f"⚠️ Entferne Spalten mit >30% fehlenden Werten: {high_missing}")
    df.drop(columns=high_missing, inplace=True)
    df.fillna(df.median(numeric_only=True), inplace=True)

    # 🔹 7. Spaltennamen bereinigen
    df.columns = (df.columns
        .str.strip()
        .str.replace(r"\s+", "_", regex=True)  # Ersetze Leerzeichen durch Unterstriche
        .str.replace(r"[^A-Za-z0-9_]", "", regex=True)  # Entferne Sonderzeichen
        .str.replace(r"_+", "_", regex=True)  # Ersetze mehrere Unterstriche durch einen
    )

    print("✅ Spaltennamen bereinigt!")

    print("\n✅ Datenbereinigung abgeschlossen!")
    return df

# 🔹 Bereinigte Daten abrufen
df = clean_data(df)




🔷 Starte Datenbereinigung...
✅ Erfolgreich konvertierte Spalten: ['Patient ID', 'Age', 'Body Mass Index (Kg/m2)', 'Albumin (g/L)', 'Normalized Troponin', 'Total Cholesterol (mmol/L)', 'Gamma-glutamil transpeptidase (IU/L)', 'Glucose (mmol/L)', 'Hemoglobin (g/L)', 'HDL (mmol/L)', 'Potassium (mEq/L)', 'LDL (mmol/L)', 'Protein (g/L)', 'T3 (pg/dL)', 'T4 (ng/L)', 'Troponin (ng/mL)', 'TSH (mIU/L)', 'Urea (mg/dL)', 'Cardiothoracic ratio', 'Holter onset (hh:mm:ss)', 'Number of ventricular premature contractions per hour', 'SDNN (ms)', 'RMSSD (ms)', 'pNN50 (%)']
⚠️ Nicht konvertierte Spalten (enthielten Text): ['Patient_ID', 'Holter_onset_hhmmss']
⚠️ Entferne Spalten mit >30% fehlenden Werten: ['Exit of the study', 'T3 (pg/dL)', 'Holter onset (hh:mm:ss)', 'Longest RR pause (ms)', 'Bradycardia', 'SDNN (ms)', 'SDANN (ms)', 'RMSSD (ms)', 'pNN50 (%)']
✅ Spaltennamen bereinigt!

✅ Datenbereinigung abgeschlossen!


In [3]:
print(df.dtypes)  # Überprüfe die Datentypen aller Spalten
print(df.isnull().sum())  # Prüfe auf fehlende Werte
print(df["Age"].unique())  # Teste eine Spalte mit vorherigen Problemen


Patient_ID                              int64
Followup_period_from_enrollment_days    int64
days_4years                             int64
Cause_of_death                          int64
Age                                     int64
                                        ...  
Spironolactone_yes1                     int64
Statins_yes1                            int64
Hidralazina_yes1                        int64
ACE_inhibitor_yes1                      int64
Nitrovasodilator_yes1                   int64
Length: 94, dtype: object
Patient_ID                              0
Followup_period_from_enrollment_days    0
days_4years                             0
Cause_of_death                          0
Age                                     0
                                       ..
Spironolactone_yes1                     0
Statins_yes1                            0
Hidralazina_yes1                        0
ACE_inhibitor_yes1                      0
Nitrovasodilator_yes1                   0
Length

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 992 entries, 0 to 991
Data columns (total 94 columns):
 #   Column                                                 Non-Null Count  Dtype  
---  ------                                                 --------------  -----  
 0   Patient_ID                                             992 non-null    int64  
 1   Followup_period_from_enrollment_days                   992 non-null    int64  
 2   days_4years                                            992 non-null    int64  
 3   Cause_of_death                                         992 non-null    int64  
 4   Age                                                    992 non-null    int64  
 5   Gender_male1                                           992 non-null    int64  
 6   Weight_kg                                              992 non-null    int64  
 7   Height_cm                                              992 non-null    int64  
 8   Body_Mass_Index_Kgm2                              

In [5]:
import pandas as pd

# Prüfe die Anzahl der Spalten
num_columns = df.shape[1]

# Prüfe die Anzahl der fehlenden Werte pro Spalte
missing_values = df.isnull().sum()
missing_values_sorted = missing_values[missing_values > 0].sort_values(ascending=False)

# Liste der Spalten im DataFrame
columns_list = df.columns.tolist()

# Ergebnisse ausgeben
num_columns, missing_values_sorted, columns_list


(94,
 Series([], dtype: int64),
 ['Patient_ID',
  'Followup_period_from_enrollment_days',
  'days_4years',
  'Cause_of_death',
  'Age',
  'Gender_male1',
  'Weight_kg',
  'Height_cm',
  'Body_Mass_Index_Kgm2',
  'NYHA_class',
  'Diastolic_blood_pressure_mmHg',
  'Systolic_blood_pressure_mmHg',
  'HF_etiology_Diagnosis',
  'Diabetes_yes1',
  'History_of_dyslipemia_yes1',
  'Peripheral_vascular_disease_yes1',
  'History_of_hypertension_yes1',
  'Prior_Myocardial_Infarction_yes1',
  'Prior_implantable_device',
  'Prior_Revascularization',
  'Syncope',
  'daily_smoking_cigarretesday',
  'smokefree_time_years',
  'cigarettes_year',
  'alcohol_consumption_standard_units',
  'Albumin_gL',
  'ALT_or_GPT_IUL',
  'AST_or_GOT_IUL',
  'Normalized_Troponin',
  'Total_Cholesterol_mmolL',
  'Creatinine_molL',
  'Gammaglutamil_transpeptidase_IUL',
  'Glucose_mmolL',
  'Hemoglobin_gL',
  'HDL_mmolL',
  'Potassium_mEqL',
  'LDL_mmolL',
  'Sodium_mEqL',
  'ProBNP_ngL',
  'Protein_gL',
  'T4_ngL',
  'Trop